# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to C:\Users\Razia Khalid
[nltk_data]     Butt\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to C:\Users\Razia Khalid
[nltk_data]     Butt\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Razia Khalid
[nltk_data]     Butt\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [3]:
# import libraries
import pandas as pd
import re
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

In [23]:
# load data from database
database_filepath = '../data/DisasterResponse.db'

engine = create_engine('sqlite:///'+database_filepath)
df = pd.read_sql_table('DisasterResponse',con=engine)
X = df['message'].values
Y = df.iloc[:,4:]

### 3. Write a tokenization function to process your text data

In [24]:
def tokenize(text):
    tokens = word_tokenize(text)

    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [27]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)    

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [29]:
y_pred = pipeline.predict(X_test)

In [30]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)

y_pred_pd.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
col = list(y_test.columns)

In [50]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred]), target_names = col[1:]))

                        precision    recall  f1-score   support

               request       0.90      0.40      0.56      1125
                 offer       0.00      0.00      0.00        27
           aid_related       0.78      0.61      0.68      2749
          medical_help       0.61      0.05      0.09       536
      medical_products       0.83      0.07      0.14       324
     search_and_rescue       0.71      0.02      0.05       205
              security       0.00      0.00      0.00       128
              military       0.77      0.04      0.08       223
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.25      0.39       395
                  food       0.91      0.37      0.53       717
               shelter       0.84      0.27      0.41       567
              clothing       0.71      0.10      0.18        97
                 money       1.00      0.02      0.04       154
        missing_people       0.50      

C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [45]:
for i in range(len(col)):
    print('Category: {} '.format(col[i]))
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])))

Category: related 
              precision    recall  f1-score   support

           0       0.72      0.28      0.40      1548
           1       0.81      0.97      0.88      4958
           2       0.65      0.42      0.51        48

    accuracy                           0.80      6554
   macro avg       0.72      0.55      0.60      6554
weighted avg       0.79      0.80      0.76      6554

Accuracy 0.7985962770826975


Category: request 
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5429
           1       0.90      0.40      0.56      1125

    accuracy                           0.89      6554
   macro avg       0.90      0.70      0.75      6554
weighted avg       0.89      0.89      0.87      6554

Accuracy 0.8901434238632896


Category: offer 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy 

C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Category: food 
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5837
           1       0.91      0.37      0.53       717

    accuracy                           0.93      6554
   macro avg       0.92      0.68      0.75      6554
weighted avg       0.93      0.93      0.91      6554

Accuracy 0.9275251754653646


Category: shelter 
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5987
           1       0.84      0.27      0.41       567

    accuracy                           0.93      6554
   macro avg       0.89      0.63      0.69      6554
weighted avg       0.93      0.93      0.92      6554

Accuracy 0.932255111382362


Category: clothing 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6457
           1       0.71      0.10      0.18        97

    accuracy                           0.99      6554
   macro avg 

In [53]:
# for column in y_test.columns:    
#     print(' {}\n'.format(column))
#     print(classification_report(y_test[column],y_pred_pd[column]))

### 6. Improve your model
Use grid search to find better parameters. 

In [51]:
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],
        
        }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [52]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)


C:\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [80]:
cv.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 200}

In [58]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred]), target_names = col[1:]))

C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                        precision    recall  f1-score   support

               request       0.91      0.42      0.57      1125
                 offer       0.00      0.00      0.00        27
           aid_related       0.78      0.62      0.69      2749
          medical_help       0.63      0.04      0.08       536
      medical_products       0.80      0.07      0.14       324
     search_and_rescue       0.71      0.02      0.05       205
              security       0.00      0.00      0.00       128
              military       0.87      0.06      0.11       223
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.26      0.40       395
                  food       0.89      0.44      0.59       717
               shelter       0.85      0.31      0.45       567
              clothing       0.60      0.06      0.11        97
                 money       1.00      0.02      0.04       154
        missing_people       0.50      

C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:

- try other machine learning algorithms
- add other features besides the TF-IDF


In [63]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [72]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200)))
])

In [73]:
# cv = GridSearchCV(pipeline, param_grid = parameters)
pipeline.fit(X_train, y_train)  
y_pred = pipeline.predict(X_test)

print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred]), target_names = col[1:]))

C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


                        precision    recall  f1-score   support

               request       0.92      0.42      0.57      1125
                 offer       0.00      0.00      0.00        27
           aid_related       0.80      0.61      0.69      2749
          medical_help       0.61      0.03      0.05       536
      medical_products       0.74      0.05      0.10       324
     search_and_rescue       0.57      0.02      0.04       205
              security       1.00      0.01      0.02       128
              military       0.78      0.03      0.06       223
           child_alone       0.00      0.00      0.00         0
                 water       0.94      0.22      0.36       395
                  food       0.91      0.36      0.52       717
               shelter       0.85      0.25      0.38       567
              clothing       0.67      0.08      0.15        97
                 money       1.00      0.02      0.04       154
        missing_people       0.00      

In [ ]:
# cv.fit(X_train, y_train)
# y_pred = cv.predict(X_test)

# print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred]), target_names = col))

### 9. Export your model as a pickle file

In [ ]:
import pickle 
  
# Save the model as a pickle in a file 
pickle.dump(pipeline, open('classifier.pkl', 'wb'))